In [1]:
%config IPCompleter.greedy=True

In [21]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import csv

data_path = "./weapon_data.csv"

In [22]:
def getCsvHeader(filename):
    with open(filename, mode='r') as f:
        reader = csv.reader(f)
        header = next(reader)
        return header
    
def getCsvAsDict(filename):
    with open(filename, mode='r') as infile:
        dictReader = csv.DictReader(infile)
        header = getCsvHeader(filename)

        final_dict = { col_name: [] for col_name in header }
        x = 0

        for row in dictReader:
            ##int(row)
            for col_name in header:
                final_dict[col_name].append(row[col_name])

        return final_dict

In [23]:
features = getCsvAsDict(data_path)
#print(features)

In [24]:
def tensorToParameterDict(tensor, cols_to_vars_dict):
    idx =  0 #keep track of the id to find the right value in the tensor
    result = dict()
    for key in cols_to_vars_dict:
        '''
        e.g., key = _NumericColumn(key='bdrop', shape=(1,), default_value=None, 
                    dtype=tf.float32, normalizer_fn=None)
        then key[0] = 'bdrop'
        '''
        if key[0] in NUMERICAL_PARAMS:
            result[key[0]] = str(tensor[idx])
        
            '''
            e.g., key = _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
            then key[0] = _VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
            and key[0][0] = 'firemode'
            and key[0][0][0] = 'Automatic'
            '''
        elif (len(key[0]) > 0) and key[0][0] in CATEGORICAL_PARAMS:
            for i in range(0, len(key[0][1])-1):
                param = key[0][0] + "_" + key[0][1][i]
                result[param] = str(tensor[idx])
                idx += 1
                
        idx += 1
    return result

In [33]:
CATEGORICAL_PARAMS  = ['type', 'firemode', 'ammo']
NUMERICAL_PARAMS = ['rof', 'damages_first', 'damages_last', 'dmg_distances_first', 'dmg_distances_last', 'initialspeed',
                    'drag', 'magsize', 'shotspershell', 'reloadempty', 'hordispersion', 'verdispersion', 
                    'hiprecoildec', 'hiprecoilright', 'hiprecoilup' ]

for key, values in features.items():
    if key in NUMERICAL_PARAMS:
        features[key] = [float('{:.4f}'.format(float(value))) for value in values]

type_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'type', ['Shotgun', 'Pistol', 'Rifle', 'Submachine Gun', 'Sniper Rifle', 'Light Machine Gun'])
type_column = tf.feature_column.indicator_column(type_column)

firemode_colum = tf.feature_column.categorical_column_with_vocabulary_list(
    'firemode', ['Automatic', 'Semi-Automatic', 'Single-Action'])
firemode_colum = tf.feature_column.indicator_column(firemode_colum)

columns = [tf.feature_column.numeric_column(param) for param in NUMERICAL_PARAMS]
columns.append(type_column)
columns.append(firemode_colum)

cols_to_vars_dict = {}
inputs = tf.feature_column.input_layer(features, columns, cols_to_vars=cols_to_vars_dict)


#print(cols_to_vars_dict)
#print(inputs)

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()

with tf.Session() as sess:
    sess.run((var_init, table_init))
    print(sess.run(inputs))
    
tf.set_random_seed(1234)
inputs = tf.random_shuffle(inputs)

with tf.Session() as sess:
    sess.run((var_init, table_init))
    print(sess.run(inputs))

[[ 7.7   3.85 12.   ...  0.    0.    1.2 ]
 [ 7.7   3.85 14.   ...  0.    0.    1.8 ]
 [ 7.7   3.85 14.   ...  0.    0.    1.2 ]
 ...
 [90.   70.    0.   ...  1.    0.    0.  ]
 [90.   70.    0.   ...  1.    0.    0.  ]
 [23.   13.5   6.   ...  0.    0.    0.  ]]
[[26.5 13.5  6.  ...  0.   0.   0. ]
 [40.  35.  40.  ...  0.   0.   0. ]
 [28.  21.  11.  ...  0.   1.   0. ]
 ...
 [26.5 23.  11.  ...  0.   0.   0. ]
 [28.  21.  11.  ...  0.   1.   0. ]
 [26.5 20.  11.  ...  0.   1.   0. ]]


In [34]:
with tf.Session() as sess:
    sess.run((var_init, table_init))
    
    processedInput = sess.run(inputs)
    print("input size = %i" %len(processedInput))
    print(processedInput)
    tensorDict = tensorToParameterDict(processedInput[len(processedInput)-1], cols_to_vars_dict)
    for key, value in tensorDict.items():
        print(key + " = " + value)

input size = 148
[[26.5 13.5  6.  ...  0.   0.   0. ]
 [40.  35.  40.  ...  0.   0.   0. ]
 [28.  21.  11.  ...  0.   1.   0. ]
 ...
 [26.5 23.  11.  ...  0.   0.   0. ]
 [28.  21.  11.  ...  0.   1.   0. ]
 [26.5 20.  11.  ...  0.   1.   0. ]]
damages_first = 26.5
damages_last = 20.0
dmg_distances_first = 11.0
dmg_distances_last = 50.0
drag = 0.0025
firemode_Automatic = 1.0
firemode_Semi-Automatic = 0.0
hiprecoildec = 6.0
hiprecoilright = 0.1232
hiprecoilup = 0.28
hipstandmovemax = 4.25
hipstandmovespreaddec = 9.0
hipstandmovespreadinc = -0.18
hordispersion = 0.0
initialspeed = 740.0
magsize = 26.0
reloadempty = 4.25
rof = 475.0
shotspershell = 1.0
type_Shotgun = 0.0
type_Pistol = 0.0
type_Rifle = 0.0
type_Submachine Gun = 0.0
type_Sniper Rifle = 0.0
verdispersion = 0.0
